## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [91]:
import pandas as pd
import numpy as np
import math

from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tr_mcc_codes_path = '/content/drive/MyDrive/Data/tr_mcc_codes.csv'
tr_types_path = '/content/drive/MyDrive/Data/tr_types.csv'
gender_train_path = '/content/drive/MyDrive/Data/gender_train.csv'
transaction_path = '/content/drive/MyDrive/Data/transactions.csv'

tr_mcc_codes_df = pd.read_csv(tr_mcc_codes_path, sep = ';')
tr_types_df = pd.read_csv(tr_types_path, sep = ";")
gender_train_df = pd.read_csv(gender_train_path, sep = ",")
transaction_df = pd.read_csv(transaction_path, sep = ',', nrows = 1000000)


###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:
random_transactions = transaction_df['tr_type'].sample(n = 1000)

temp_df = pd.merge(random_transactions, tr_types_df, on = 'tr_type')
temp_df = temp_df[temp_df['tr_description'].str.contains('ATM|POS')]
temp_df

,tr_type,tr_description
454,1210,Покупка. POS Зарубеж. ТУ
455,1210,Покупка. POS Зарубеж. ТУ
456,1210,Покупка. POS Зарубеж. ТУ
457,1210,Покупка. POS Зарубеж. ТУ
458,1210,Покупка. POS Зарубеж. ТУ
...,...,...
986,7020,Взнос наличных через POS
987,7020,Взнос наличных через POS
995,2320,Безналичный перевод денежных средств через POS
996,2320,Безналичный перевод денежных средств через POS


In [ ]:
print(len(temp_df) / 1000)

0.381


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
# 3.1
trans_df = transaction_df
trans_df['tr_type'].value_counts()


1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
         ...  
2446         4
4096         4
8146         3
1510         2
8100         2
Name: tr_type, Length: 72, dtype: int64

In [ ]:
# 3.2
top10list = trans_df['tr_type'].value_counts()[:10].index.to_list()
print(top10list)

[1010, 2010, 7070, 1110, 1030, 2370, 7010, 7030, 7071, 1100]


In [ ]:
# 3.2 (tr_descriptions)

for tr_type_temp in top10list:
  print('For tr_type: ', tr_type_temp, "tr_description is ", transaction_df.loc[transaction_df['tr_type'] == tr_type_temp, 'tr_description'].iloc[0])

# df.loc[df['B'] == 3, 'A'].iloc[0]  stackoverflow <З

For tr_type:  1010 tr_description is  Покупка. POS ТУ СБ РФ
For tr_type:  2010 tr_description is  Выдача наличных в АТМ Сбербанк России
For tr_type:  7070 tr_description is  Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя)
For tr_type:  1110 tr_description is  Покупка. POS ТУ Россия
For tr_type:  1030 tr_description is  Оплата услуги. Банкоматы СБ РФ
For tr_type:  2370 tr_description is  Списание с карты на карту по операции <перевода с карты на карту> через Мобильный банк (без комиссии)
For tr_type:  7010 tr_description is  Взнос наличных через АТМ (в своем тер.банке)
For tr_type:  7030 tr_description is  Перевод на карту (с карты) через АТМ (в пределах одного тер.банка)
For tr_type:  7071 tr_description is  Перевод на карту (с карты) через Мобильный банк (с взиманием комиссии с отправителя)
For tr_type:  1100 tr_description is  Покупка. ТУ  Россия


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
income_df = transaction_df.loc[transaction_df['amount'] > 0]
income_df = income_df[['customer_id','amount']]

outcome_df = transaction_df.loc[transaction_df['amount'] <= 0]
outcome_df = outcome_df[['customer_id','amount']]

all_customer_income_df = income_df.groupby('customer_id').sum()
all_customer_outcome_df = outcome_df.groupby('customer_id').sum()

all_customer_income_df = all_customer_income_df.sort_values(by = ['amount'], ascending = False)
all_customer_outcome_df = all_customer_outcome_df.sort_values(by = ['amount'])

income_max = all_customer_income_df['amount'].to_list()[0]
outcome_max = all_customer_outcome_df['amount'].to_list()[0]

# 3.1 && 3.2
print("Maximum income and outcome respectively: ")
print(income_max, outcome_max)

max_customer_income_id = all_customer_income_df.index[all_customer_income_df['amount'] == income_max].to_list()
max_customer_outcome_id = all_customer_outcome_df.index[all_customer_outcome_df['amount'] == outcome_max].to_list()

print("Maximum income and outcome customers' ids respectively:")
print(max_customer_income_id, max_customer_outcome_id)

# 3.3
print("Diff: ")
print(income_max + outcome_max)





Maximum income and outcome respectively: 
1248114886.81 -1249952204.79
Maximum income and outcome customers' ids respectively:
[70780820] [70780820]
Diff: 
-1837317.980000019


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
# 4.1
top_transactions_by_amount = transaction_df['tr_type'].value_counts()
top_transactions_by_amount = top_transactions_by_amount.sort_values(ascending = False).head(10)

for tr_type in top_transactions_by_amount.index:
  current_tr_type_transactions_df = transaction_df[transaction_df['tr_type'] == tr_type]
  current_tr_type_median = current_tr_type_transactions_df['amount'].median()
  print('median for transaction type ', tr_type, ' is: ', current_tr_type_median)
  current_tr_type_mean = current_tr_type_transactions_df['amount'].mean()
  print('mean for transaction type ', tr_type, ' is: ', current_tr_type_mean)
  print('\n')

median for transaction type  1010  is:  -7411.52
mean for transaction type  1010  is:  -19784.748639648315


median for transaction type  2010  is:  -44918.32
mean for transaction type  2010  is:  -136077.62932505234


median for transaction type  7070  is:  11319.42
mean for transaction type  7070  is:  65569.83170018934


median for transaction type  1110  is:  -11207.57
mean for transaction type  1110  is:  -32119.33037077222


median for transaction type  1030  is:  -2245.92
mean for transaction type  1030  is:  -5320.9802218113155


median for transaction type  2370  is:  -44918.32
mean for transaction type  2370  is:  -205418.24903167944


median for transaction type  7010  is:  112295.79
mean for transaction type  7010  is:  276391.7895961331


median for transaction type  7030  is:  13951.52
mean for transaction type  7030  is:  86104.33290896451


median for transaction type  7071  is:  3593.47
mean for transaction type  7071  is:  66806.82662251407


median for transaction ty

In [89]:
# 4.2 (Max income customer)

max_income_customer = transaction_df[transaction_df['customer_id'] == max_customer_income_id[0]]
max_income_customer_types = max_income_customer['tr_type'].unique()

print('Medians and means for customer with a max income: \n')
for tr_type_temp in max_income_customer_types:
  curr_tr_type_median = max_income_customer[max_income_customer['tr_type'] == tr_type_temp]['amount'].median()
  print('For tr_type: ', tr_type_temp, "median is ", curr_tr_type_median)
  curr_tr_type_mean = max_income_customer[max_income_customer['tr_type'] == tr_type_temp]['amount'].mean()
  print('For tr_type: ', tr_type_temp, "mean is ", curr_tr_type_mean, '\n')

Medians and means for customer with a max income: 

For tr_type:  2010 median is  -3368873.66
For tr_type:  2010 mean is  -2941055.8709415505 

For tr_type:  2330 median is  -2245915.77
For tr_type:  2330 mean is  -2382398.344615385 

For tr_type:  2370 median is  -2245915.77
For tr_type:  2370 mean is  -2218504.987086618 

For tr_type:  6110 median is  10028.01
For tr_type:  6110 mean is  17562.93173076922 

For tr_type:  7020 median is  10319.985
For tr_type:  7020 mean is  14651.290741626804 

For tr_type:  7070 median is  8714.15
For tr_type:  7070 mean is  13848.205232082266 

For tr_type:  7030 median is  8848.91
For tr_type:  7030 mean is  14061.955241583857 

For tr_type:  7071 median is  12543.439999999999
For tr_type:  7071 mean is  18600.572007692284 

For tr_type:  7021 median is  28500.67
For tr_type:  7021 mean is  28500.67 

For tr_type:  7031 median is  12992.619999999999
For tr_type:  7031 mean is  19210.358452380955 

For tr_type:  7074 median is  9387.93
For tr_type:

In [90]:
# 4.2 (Max outcome customer)
max_outcome_customer = transaction_df[transaction_df['customer_id'] == max_customer_outcome_id[0]]
max_outcome_customer_types = max_outcome_customer['tr_type'].unique()

print('Medians and means for customer with a max outcome: \n')
for tr_type_temp in max_outcome_customer_types:
  curr_tr_type_median = max_outcome_customer[max_outcome_customer['tr_type'] == tr_type_temp]['amount'].median()
  print('For tr_type: ', tr_type_temp, "median is ", curr_tr_type_median)
  curr_tr_type_mean = max_outcome_customer[max_outcome_customer['tr_type'] == tr_type_temp]['amount'].mean()
  print('For tr_type: ', tr_type_temp, "mean is ", curr_tr_type_mean, '\n')



Medians and means for customer with a max outcome: 

For tr_type:  2010 median is  -3368873.66
For tr_type:  2010 mean is  -2941055.8709415505 

For tr_type:  2330 median is  -2245915.77
For tr_type:  2330 mean is  -2382398.344615385 

For tr_type:  2370 median is  -2245915.77
For tr_type:  2370 mean is  -2218504.987086618 

For tr_type:  6110 median is  10028.01
For tr_type:  6110 mean is  17562.93173076922 

For tr_type:  7020 median is  10319.985
For tr_type:  7020 mean is  14651.290741626804 

For tr_type:  7070 median is  8714.15
For tr_type:  7070 mean is  13848.205232082266 

For tr_type:  7030 median is  8848.91
For tr_type:  7030 mean is  14061.955241583857 

For tr_type:  7071 median is  12543.439999999999
For tr_type:  7071 mean is  18600.572007692284 

For tr_type:  7021 median is  28500.67
For tr_type:  7021 mean is  28500.67 

For tr_type:  7031 median is  12992.619999999999
For tr_type:  7031 mean is  19210.358452380955 

For tr_type:  7074 median is  9387.93
For tr_type

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transaction_df = pd.merge(transaction_df, gender_train_df, how='left')
transaction_df = pd.merge(transaction_df, tr_mcc_codes_df, how='inner')
transaction_df = pd.merge(transaction_df, tr_types_df, how='inner')
transaction_df.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [92]:
# 5.1
gender_number_zerro = transaction_df[(transaction_df['gender'] == 0) & (transaction_df['amount'] < 0)]
gender_number_one = transaction_df[(transaction_df['gender'] == 1) & (transaction_df['amount'] < 0)]

print('module out of mean for outcome: ', math.fabs(math.fabs(gender_number_zerro['amount'].mean()) - math.fabs(gender_number_one['amount'].mean())))

module out of mean for outcome:  32718.05492021877


In [93]:
# 5.2

gender_number_zerro = transaction_df[(transaction_df['gender'] == 0) & (transaction_df['amount'] > 0)]
gender_number_one = transaction_df[(transaction_df['gender'] == 1) & (transaction_df['amount'] > 0)]

print('module out of mean for outcome: ', math.fabs(math.fabs(gender_number_zerro['amount'].mean()) - math.fabs(gender_number_one['amount'].mean())))

module out of mean for outcome:  63366.57104801515


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [109]:
zerro_curr_transactions = pd.DataFrame([])
one_curr_transactions = pd.DataFrame([])

for tr_type in tr_types_df['tr_type']:

  zerro = transaction_df[(transaction_df['tr_type'] == tr_type) & (transaction_df['gender'] == 0) & (transaction_df['amount'] > 0)]
  one = transaction_df[(transaction_df['tr_type'] == tr_type) & (transaction_df['gender'] == 1) & (transaction_df['amount'] > 0)]

  if zerro.size > 0:
    zerro_curr_index = zerro['amount'].idxmax()
    zerro_curr_transactions = zerro_curr_transactions.append(transaction_df.loc[zerro_curr_index])

  if one.size > 0:
    one_curr_index = one['amount'].idxmax()
    one_curr_transactions = one_curr_transactions.append(transaction_df.loc[one_curr_index])

zerro_top = zerro_curr_transactions['amount'].sort_values().head(10)
one_top = one_curr_transactions['amount'].sort_values().head(10)

print('Top minimum by max_income for the gender number zerro: ')
print(zerro_top, '\n')

print('Top minimum by max_income for the gender number one: ')
print(one_top)


Top minimum by max_income for the gender number zerro: 
782598     1122.96
715900     2245.92
713566     2245.92
752472     2335.75
718156     5041.86
998258     6737.75
714837    22459.16
321089    24839.83
669411    44918.32
716363    68315.82
Name: amount, dtype: float64 

Top minimum by max_income for the gender number one: 
713615      2245.92
802537      2245.92
386903     28803.87
518893     64682.37
999567     79280.83
998278     87590.72
638141     89836.63
714995    112295.79
755215    113194.15
718186    121279.45
Name: amount, dtype: float64


In [110]:
zerro_top = zerro_curr_transactions.sort_values(by = ['amount']).head(10)
one_top = one_curr_transactions.sort_values(by = ['amount']).head(10)

zerro_tr_types = zerro_top['tr_type'].unique()
one_tr_types = one_top['tr_type'].unique()

the_same_types = pd.merge(zerro_top, one_top, on='tr_type', how='inner')
the_same_types[['tr_type', 'amount_x','amount_y']]

,tr_type,amount_x,amount_y
0,4110.0,2245.92,2245.92
1,2370.0,2335.75,113194.15
2,2110.0,22459.16,112295.79
3,1010.0,24839.83,28803.87
4,2010.0,44918.32,89836.63


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 